In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

# Load & prep data

In [21]:
path = 'data/eng-fra.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [22]:
with open(path, 'r') as f:
    lines = f.read().split('\n')

In [23]:
for line in lines[:min(10000, len(lines) - 1)]:
    # tab is the start token, new line is the end token
    input_text, target_text = line.split('\t')
    input_texts.append(input_text)
    
    target_text = '\t' + target_text + '\n'
    target_texts.append(target_text)
    
    [input_characters.add(char) for char in input_text]
    [target_characters.add(char) for char in target_text]

In [26]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_len = max([len(text) for text in input_texts])
max_decoder_seq_len = max([len(text) for text in target_texts])

In [28]:
print('Number of samples: {}'.format(len(input_texts)))
print('Number of unique input tokens: {}'.format(num_encoder_tokens))
print('Number of unique output tokens: {}'.format(num_decoder_tokens))
print('Max sequence length for inputs: {}'.format(max_encoder_seq_len))
print('Max sequence length for outputs: {}'.format(max_decoder_seq_len))

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 63


In [29]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [31]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_len, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')

In [32]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Build model

In [35]:
# input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [36]:
# output sequence
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [47]:
# define model
mdl = Model([encoder_inputs, decoder_inputs], decoder_outputs)
mdl.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Training model

In [48]:
mdl.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=1, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 59s 7ms/step - loss: 0.4656 - val_loss: 0.5792


In [49]:
mdl.optimizer.lr = 0.1

In [50]:
# run 5 epochs with the increased learning rate
mdl.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=5, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.4390 - val_loss: 0.5540
Epoch 2/5
8000/8000 [==============================] - 61s 8ms/step - loss: 0.4174 - val_loss: 0.5455
Epoch 3/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.4000 - val_loss: 0.5293
Epoch 4/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.3839 - val_loss: 0.5252
Epoch 5/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.3693 - val_loss: 0.5087


In [51]:
mdl.optimizer.lr = 0.01

In [52]:
# run another 5 epochs with the decreased learning rate
mdl.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=5, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 59s 7ms/step - loss: 0.3560 - val_loss: 0.4993
Epoch 2/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.3434 - val_loss: 0.4902
Epoch 3/5
8000/8000 [==============================] - 59s 7ms/step - loss: 0.3318 - val_loss: 0.4891
Epoch 4/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.3204 - val_loss: 0.4780
Epoch 5/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.3098 - val_loss: 0.4770


In [53]:
mdl.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=5, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 57s 7ms/step - loss: 0.2999 - val_loss: 0.4721
Epoch 2/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.2903 - val_loss: 0.4746
Epoch 3/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.2809 - val_loss: 0.4668
Epoch 4/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.2725 - val_loss: 0.4672
Epoch 5/5
8000/8000 [==============================] - 58s 7ms/step - loss: 0.2640 - val_loss: 0.4648


# Inference

In [54]:
encoder_mdl = Model(encoder_inputs, encoder_states)

In [55]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_mdl = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [63]:
# decode sequences back to characters
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encode input as state vectors.
    states_value = encoder_mdl.predict(input_seq)

    # generate empty target sequence & prepend with start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop = False
    decoded_sentence = ''
    while not stop:
        output_tokens, h, c = decoder_mdl.predict(
            [target_seq] + states_value)

        # sample token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # if max length or stop character, stop
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_len:
            stop = True

        # update target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # update states
        states_value = [h, c]

    return decoded_sentence

In [64]:
for seq_idx in range(100):
    input_seq = encoder_input_data[seq_idx:seq_idx+1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("---------------------")
    print("Input: {}".format(input_texts[seq_idx]))
    print("Decoded: {}".format(decoded_sentence))

---------------------
Input: Go.
Decoded: Allez !

---------------------
Input: Run!
Decoded: Rette de somber !

---------------------
Input: Run!
Decoded: Rette de somber !

---------------------
Input: Wow!
Decoded: Fais-le demander !

---------------------
Input: Fire!
Decoded: Reste partir !

---------------------
Input: Help!
Decoded: Rettez !

---------------------
Input: Jump.
Decoded: Va te parder !

---------------------
Input: Stop!
Decoded: Arrête de trainer !

---------------------
Input: Stop!
Decoded: Arrête de trainer !

---------------------
Input: Stop!
Decoded: Arrête de trainer !

---------------------
Input: Wait!
Decoded: Reste pas tomber !

---------------------
Input: Wait!
Decoded: Reste pas tomber !

---------------------
Input: I see.
Decoded: Je l'ai vue.

---------------------
Input: I try.
Decoded: Je l'ai contré.

---------------------
Input: I won!
Decoded: Je l'ai contrarié.

---------------------
Input: I won!
Decoded: Je l'ai contrarié.

--------------